
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio de Sistemas de Recomendación

## Objetivos del Laboratorio

1.	Implementar y comparar diferentes tipos de sistemas de recomendación utilizando datos reales.

2.	Explorar el impacto de las características de los datos en la calidad de las recomendaciones.

3.	Desarrollar habilidades en preprocesamiento, modelado y evaluación de sistemas de recomendación.


## Los problemas

A lo largo de este laboratorio deberás enfrentarte a distintos casos prácticos:

- **Caso 2: Sistema de Recomendación Basado en Popularidad**

	Debes un sistema que recomiende los juegos más populares según su calificación general y número de reseñas. Para eso deberás:
	
	- Limpiar y convertir las columnas relacionadas con las calificaciones y las reseñas en datos numéricos.
	
	- Identificar los juegos más populares utilizando métricas como la mediana de calificaciones ponderadas.
	
	- Crear una tabla de recomendaciones generales para diferentes géneros.

## Los Datos

- **2. games_ranking.csv**: Este archivo contiene información sobre el ranking de videojuegos basados en diferentes métricas. Las columnsa que tenemos son:


	- **`game_name`**: Nombre del videojuego.

	- **`genre`**: Género principal del videojuego.

	- **`rank_type`**: Tipo de ranking (por ejemplo, ventas, popularidad).

	- **`rank`**: Posición del videojuego en el ranking correspondiente.

----

- Los datos del archivo `games_ranking.csv` son ideales para implementar sistemas basados en popularidad.


In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Para visualización de datos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append("../")

# Para cargar los datos
# -----------------------------------------------------------------------
import pickle

# Para sistemas de recomendacion basados en cotenido
# -----------------------------------------------------------------------
from scipy.spatial.distance import pdist , squareform   # Para calcular la distancia entre puntos 

# Importamos nuestras funciones
# -----------------------------------------------------------------------
from src import soporte_sistemas_recomendacion as sr

# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.0f}'.format
%load_ext autoreload
%autoreload 2

In [2]:
# cargamos los datos
df_rat = pd.read_csv('../data/games_ranking.csv').set_index('game_name')

# antes de nada tenemos que transponer el conjunto de datos 
df_rat.head()

,genre,rank_type,rank
game_name,,,
Counter-Strike 2,Action,Sales,1
"Warhammer 40,000: Space Marine 2",Action,Sales,2
Cyberpunk 2077,Action,Sales,3
Black Myth: Wukong,Action,Sales,4
ELDEN RING,Action,Sales,5


In [3]:
# Filtrar solo el índice y la columna 'rank'
df_rat = df_rat[['rank']]

In [4]:
df_rat.head()

,rank
game_name,
Counter-Strike 2,1
"Warhammer 40,000: Space Marine 2",2
Cyberpunk 2077,3
Black Myth: Wukong,4
ELDEN RING,5


In [5]:
# calculamos las distancias usando squareform
distancias_square = squareform(pdist(df_rat, 'euclidean'))   

# pertenecen al intervalo (0, 1) de esta forma conseguimos que los valores este acotados 
similitud_square = 1/(1 + distancias_square)
similitud_square[:10]

array([[1.        , 0.5       , 0.33333333, ..., 0.02631579, 0.02564103,
        0.025     ],
       [0.5       , 1.        , 0.5       , ..., 0.02702703, 0.02631579,
        0.02564103],
       [0.33333333, 0.5       , 1.        , ..., 0.02777778, 0.02702703,
        0.02631579],
       ...,
       [0.125     , 0.14285714, 0.16666667, ..., 0.03225806, 0.03125   ,
        0.03030303],
       [0.11111111, 0.125     , 0.14285714, ..., 0.03333333, 0.03225806,
        0.03125   ],
       [0.1       , 0.11111111, 0.125     , ..., 0.03448276, 0.03333333,
        0.03225806]])

In [6]:
df_similitud = pd.DataFrame(similitud_square, columns=df_rat.T.columns, index=df_rat.T.columns)
df_similitud.head(2)

game_name,Counter-Strike 2,"Warhammer 40,000: Space Marine 2",Cyberpunk 2077,Black Myth: Wukong,ELDEN RING,PUBG: BATTLEGROUNDS,DRAGON BALL: Sparking! ZERO,Apex Legends™,Dota 2,Party Animals,Total War: WARHAMMER III,Cyberpunk 2077: Phantom Liberty,Once Human,NARAKA: BLADEPOINT,Warframe,Grounded,Grand Theft Auto V,Tom Clancy's Rainbow Six® Siege,Call of Duty®: Black Ops III,Like a Dragon: Infinite Wealth,Dead by Daylight,Call of Duty®,The First Descendant,Street Fighter™ 6,Call of Duty®: Modern Warfare® III - BlackCell (Season 6),The Crew™ 2,Battlefield™ V,Sea of Thieves: 2024 Edition,Halo: The Master Chief Collection,Call of Duty®: Modern Warfare® III,The Witcher 3: Wild Hunt,LOCKDOWN Protocol,Call of Duty®: Warzone™,Rust,Dead Rising Deluxe Remaster,Noita,"Warhammer 40,000: Space Marine 2",Black Myth: Wukong,Counter-Strike 2,ELDEN RING,PUBG: BATTLEGROUNDS,Cyberpunk 2077,Warframe,Dota 2,Apex Legends™,Grand Theft Auto V,Once Human,NARAKA: BLADEPOINT,Tom Clancy's Rainbow Six® Siege,The First Descendant,Call of Duty®: Black Ops III,Cyberpunk 2077: Phantom Liberty,Dead by Daylight,ELDEN RING Shadow of the Erdtree,Total War: WARHAMMER III,Call of Duty®,Street Fighter™ 6,Sea of Thieves: 2024 Edition,LOCKDOWN Protocol,The Crew™ 2,Dead Rising Deluxe Remaster,Rust,Party Animals,Call of Duty®: Modern Warfare® III - BlackCell (Season 6),Diablo® IV,FINAL FANTASY XIV Online,Destiny 2,Like a Dragon: Infinite Wealth,Wallpaper Engine,Team Fortress 2,The Elder Scrolls® Online,Red Dead Redemption 2,Black Desert,DRAGON BALL: Sparking! ZERO,Call of Duty®: Black Ops II,Risk of Rain 2,Counter-Strike 2,Grand Theft Auto V,Dota 2,Terraria,Black Myth: Wukong,Wallpaper Engine,Tom Clancy's Rainbow Six® Siege,Left 4 Dead 2,The Witcher 3: Wild Hunt,Team Fortress 2,Rust,ELDEN RING,Red Dead Redemption 2,Warframe,The Forest,Among Us,Portal 2,Cyberpunk 2077,Sekiro™: Shadows Die Twice - GOTY Edition,PAYDAY 2,Dying Light,Hades,Counter-Strike,Hollow Knight,Unturned,Destiny 2,Deep Rock Galactic,Valheim,Borderlands 2,Titanfall® 2,ARK: Survival Evolved,Vampire Survivors,Dead by Daylight,Devil May Cry 5,Monster Hunter: World,Sea of Thieves: 2024 Edition,Half-Life 2,Tomb Raider,Resident Evil 4,Counter-Strike: Source,"Warhammer 40,000: Space Marine 2",Black Myth: Wukong,Satisfactory,Once Human,Baldur's Gate 3,Persona 3 Reload,Grounded,Shin Megami Tensei V: Vengeance,Grand Theft Auto V,Like a Dragon: Infinite Wealth,The First Descendant,Street Fighter™ 6,Sea of Thieves: 2024 Edition,The Witcher 3: Wild Hunt,Rust,Dead Rising Deluxe Remaster,Microsoft Flight Simulator 40th Anniversary Edition,Dragon's Dogma 2,Lies of P,UFO 50,FINAL FANTASY VII REMAKE INTERGRADE,Black Desert,FINAL FANTASY XIV Online,Gunfire Reborn,Core Keeper,STAR WARS Jedi: Survivor™,DEATH STRANDING DIRECTOR'S CUT,Resident Evil 4,Palworld,Little Nightmares II,Little Nightmares,Sonic Frontiers,"Warhammer 40,000: Space Marine 2 - Season Pass",Alien: Isolation,Lost Judgment,ELDEN RING Shadow of the Erdtree,Lethal Company,The Elder Scrolls® Online,Thank Goodness You're Here!,STAR WARS™ Battlefront™ II,"Warhammer 40,000: Space Marine 2",Black Myth: Wukong,Baldur's Gate 3,Satisfactory,Once Human,Grand Theft Auto V,Persona 3 Reload,Sea of Thieves: 2024 Edition,The First Descendant,Rust,Street Fighter™ 6,Dead Rising Deluxe Remaster,Shin Megami Tensei V: Vengeance,FINAL FANTASY XIV Online,Like a Dragon: Infinite Wealth,Core Keeper,The Elder Scrolls® Online,Red Dead Redemption 2,Microsoft Flight Simulator 40th Anniversary Edition,Black Desert,The Witcher 3: Wild Hunt,Chained Together,Dragon's Dogma 2,Grounded,Forza Horizon 5,Gunfire Reborn,Palworld,"Warhammer 40,000: Space Marine 2 - Season Pass",ARK: Survival Ascended,ELDEN RING Shadow of the Erdtree,Lethal Company,Sons Of The Forest,FINAL FANTASY VII REMAKE INTERGRADE,Lies of P,Resident Evil 4,UFO 50,DREDGE,Crime Scene Cleaner,Thank Goodness You're Here!,STAR WARS Jedi: Survivor™,Terraria,Grand Theft Auto V,The Witcher 3: Wild Hunt,Baldur's Gate 3,Black Myth: Wukon

In [7]:
# Volvemos a poner el game en el indice para la pivot table
df_rat1 = df_rat.reset_index()
df_rat1.head()


,game_name,rank
0,Counter-Strike 2,1
1,"Warhammer 40,000: Space Marine 2",2
2,Cyberpunk 2077,3
3,Black Myth: Wukong,4
4,ELDEN RING,5


In [8]:
# pivitamos la tabla
gameRatings = df_rat1.pivot_table(index=['game_name'],columns=['game_name'],values='rank').fillna(0)
gameRatings.head()


game_name,69 Balls,7 Days to Die,ARK: Survival Ascended,ARK: Survival Evolved,Age of Empires II (Retired),Age of Empires II: Definitive Edition,Age of Empires IV: Anniversary Edition,Age of Mythology: Retold,Age of Wonders 4,Aimlabs,Ale & Tale Tavern,Alien: Isolation,All In One Sports - Hockey DLC,All In One Sports - KungFu DLC,All In One Sports - Soccer DLC,All-In-One Sports VR,American Truck Simulator,American Truck Simulator - Arkansas,Among Us,Anno 1800,Apex Legends™,Ara: History Untold,Arma 3,Armello,Assetto Corsa,Assetto Corsa Competizione,Assetto Corsa Competizione - 24H Nürburgring Pack,Assetto Corsa Competizione - GT2 Pack,Automobilista 2,Balatro,Baldur's Gate 3,Battlefield™ V,BeamNG.drive,Black Desert,Black Myth: Wukong,Bloons TD 6,Borderlands 2,Call of Duty®,Call of Duty®: Black Ops II,Call of Duty®: Black Ops III,Call of Duty®: Modern Warfare® III,Call of Duty®: Modern Warfare® III - BlackCell (Season 6),Call of Duty®: Warzone™,Car Mechanic Simulator 2021,CarX Drift Racing Online,CarX Street,CarX Street - Deluxe Cars,Chained Together,Cities: Skylines,Cities: Skylines II,Company of Heroes 3,Core Keeper,Counter-Strike,Counter-Strike 2,Counter-Strike: Source,Crime Scene Cleaner,Crusader Kings III,Crusader Kings III: Tours & Tournaments,Cyberpunk 2077,Cyberpunk 2077: Phantom Liberty,DARK SOULS™ III,DAVE THE DIVER,DCS World Steam Edition,DEATH STRANDING DIRECTOR'S CUT,DRAGON BALL: Sparking! ZERO,DREDGE,Dale & Dawson Stationery Supplies,Dark and Darker,Dead Rising Deluxe Remaster,Dead by Daylight,Deep Rock Galactic,Descenders,Destiny 2,Devil May Cry 5,DiRT Rally 2.0,Diablo® IV,Divinity: Original Sin 2 - Definitive Edition,Dorfromantik,Dota 2,Downhill Pro Racer,Dragon's Dogma 2,Driftwood,Dune: Imperium,Dying Light,Dyson Sphere Program,EA SPORTS FC™ 24,EA SPORTS FC™ 25,EA SPORTS™ Madden NFL 25,EA SPORTS™ WRC,ELDEN RING,ELDEN RING Shadow of the Erdtree,ENDLESS™ Legend,Euro Truck Simulator 2,Europa Universalis IV,F1® 24,F1® Manager 2024,FINAL FANTASY VII REMAKE INTERGRADE,FINAL FANTASY XIV Online,Fallout 4,Fallout 76,Fallout: New Vegas,Farming Simulator 25,Fields of Mistria,Fishing Planet,Football Manager 2024,For The King,Forza Horizon 5,Frostpunk,Frostpunk 2,GP Bikes,Game Dev Tycoon,Garry's Mod,God of War,Golf Gang,Golf It!,Golf Peaks,Golf With Your Friends,Grand Mountain Adventure: Wonderlands,Grand Theft Auto V,Grounded,Gunfire Reborn,Gym Simulator 24,Hades,Half-Life 2,Halo: The Master Chief Collection,Hearts of Iron IV,Hogwarts Legacy,Hollow Knight,House Flipper,Inscryption,Jurassic World Evolution 2,Kenshi,Kerbal Space Program,LOCKDOWN Protocol,Left 4 Dead 2,Lethal Company,Lies of P,Liftoff®: FPV Drone Racing,Like a Dragon: Infinite Wealth,Limbus Company,Little Nightmares,Little Nightmares II,Lonely Mountains: Downhill,Lost Judgment,MARVEL SNAP,MX vs ATV All Out,MX vs ATV Legends,MX vs. ATV Reflex,MXGP - The Official Motocross Videogame,MXGP 2019 - The Official Motocross Videogame,MXGP 2021 - The Official Motocross Videogame,MXGP PRO,MXGP3 - The Official Motocross Videogame,Magic: The Gathering Arena,Manor Lords,Metaphor: ReFantazio,Microsoft Flight Simulator 40th Anniversary Edition,Monster Energy Supercross - The Official Videogame,Monster Hunter: World,Motordoom,Mount & Blade II: Bannerlord,Mount & Blade: Warband,My Summer Car,NARAKA: BLADEPOINT,NBA 2K24,NBA 2K25,NBA 2K25 Hall of Fame Pass: Season 1,Need for Speed™ Heat,Need for Speed™ Payback,Need for Speed™ Unbound,Noita,Northgard,OMORI,Once Human,Oxygen Not Included,PAYDAY 2,PGA TOUR 2K23,PIPE by BMX Streets,PUBG: BATTLEGROUNDS,Palworld,"Papers, Please",Party Animals,Path of Exile,People Playground,Persona 3 Reload,Persona 4 Golden,Persona 5 Royal,Plague Inc: Evolved,Planet Coaster 2,Plants vs. Zombies GOTY Edition,Portal 2,PowerWash Simulator,Project Zomboid,RIDE 3,RIDE 4,RIDE 5,Red Dead Redemption 2,Resident Evil 4,Ride 2,Riders Republic,RimWorld,Risk of Rain 2,Rooftops & Alleys: The Parkour Game,Russian Fishing 4,Rust,STAR WARS Jedi: Survivor™,STAR WARS™ Battl

In [9]:
# calculamos la matriz de correlacion
matr_corre = gameRatings.corr(method='pearson')
matr_corre.head()

game_name,69 Balls,7 Days to Die,ARK: Survival Ascended,ARK: Survival Evolved,Age of Empires II (Retired),Age of Empires II: Definitive Edition,Age of Empires IV: Anniversary Edition,Age of Mythology: Retold,Age of Wonders 4,Aimlabs,Ale & Tale Tavern,Alien: Isolation,All In One Sports - Hockey DLC,All In One Sports - KungFu DLC,All In One Sports - Soccer DLC,All-In-One Sports VR,American Truck Simulator,American Truck Simulator - Arkansas,Among Us,Anno 1800,Apex Legends™,Ara: History Untold,Arma 3,Armello,Assetto Corsa,Assetto Corsa Competizione,Assetto Corsa Competizione - 24H Nürburgring Pack,Assetto Corsa Competizione - GT2 Pack,Automobilista 2,Balatro,Baldur's Gate 3,Battlefield™ V,BeamNG.drive,Black Desert,Black Myth: Wukong,Bloons TD 6,Borderlands 2,Call of Duty®,Call of Duty®: Black Ops II,Call of Duty®: Black Ops III,Call of Duty®: Modern Warfare® III,Call of Duty®: Modern Warfare® III - BlackCell (Season 6),Call of Duty®: Warzone™,Car Mechanic Simulator 2021,CarX Drift Racing Online,CarX Street,CarX Street - Deluxe Cars,Chained Together,Cities: Skylines,Cities: Skylines II,Company of Heroes 3,Core Keeper,Counter-Strike,Counter-Strike 2,Counter-Strike: Source,Crime Scene Cleaner,Crusader Kings III,Crusader Kings III: Tours & Tournaments,Cyberpunk 2077,Cyberpunk 2077: Phantom Liberty,DARK SOULS™ III,DAVE THE DIVER,DCS World Steam Edition,DEATH STRANDING DIRECTOR'S CUT,DRAGON BALL: Sparking! ZERO,DREDGE,Dale & Dawson Stationery Supplies,Dark and Darker,Dead Rising Deluxe Remaster,Dead by Daylight,Deep Rock Galactic,Descenders,Destiny 2,Devil May Cry 5,DiRT Rally 2.0,Diablo® IV,Divinity: Original Sin 2 - Definitive Edition,Dorfromantik,Dota 2,Downhill Pro Racer,Dragon's Dogma 2,Driftwood,Dune: Imperium,Dying Light,Dyson Sphere Program,EA SPORTS FC™ 24,EA SPORTS FC™ 25,EA SPORTS™ Madden NFL 25,EA SPORTS™ WRC,ELDEN RING,ELDEN RING Shadow of the Erdtree,ENDLESS™ Legend,Euro Truck Simulator 2,Europa Universalis IV,F1® 24,F1® Manager 2024,FINAL FANTASY VII REMAKE INTERGRADE,FINAL FANTASY XIV Online,Fallout 4,Fallout 76,Fallout: New Vegas,Farming Simulator 25,Fields of Mistria,Fishing Planet,Football Manager 2024,For The King,Forza Horizon 5,Frostpunk,Frostpunk 2,GP Bikes,Game Dev Tycoon,Garry's Mod,God of War,Golf Gang,Golf It!,Golf Peaks,Golf With Your Friends,Grand Mountain Adventure: Wonderlands,Grand Theft Auto V,Grounded,Gunfire Reborn,Gym Simulator 24,Hades,Half-Life 2,Halo: The Master Chief Collection,Hearts of Iron IV,Hogwarts Legacy,Hollow Knight,House Flipper,Inscryption,Jurassic World Evolution 2,Kenshi,Kerbal Space Program,LOCKDOWN Protocol,Left 4 Dead 2,Lethal Company,Lies of P,Liftoff®: FPV Drone Racing,Like a Dragon: Infinite Wealth,Limbus Company,Little Nightmares,Little Nightmares II,Lonely Mountains: Downhill,Lost Judgment,MARVEL SNAP,MX vs ATV All Out,MX vs ATV Legends,MX vs. ATV Reflex,MXGP - The Official Motocross Videogame,MXGP 2019 - The Official Motocross Videogame,MXGP 2021 - The Official Motocross Videogame,MXGP PRO,MXGP3 - The Official Motocross Videogame,Magic: The Gathering Arena,Manor Lords,Metaphor: ReFantazio,Microsoft Flight Simulator 40th Anniversary Edition,Monster Energy Supercross - The Official Videogame,Monster Hunter: World,Motordoom,Mount & Blade II: Bannerlord,Mount & Blade: Warband,My Summer Car,NARAKA: BLADEPOINT,NBA 2K24,NBA 2K25,NBA 2K25 Hall of Fame Pass: Season 1,Need for Speed™ Heat,Need for Speed™ Payback,Need for Speed™ Unbound,Noita,Northgard,OMORI,Once Human,Oxygen Not Included,PAYDAY 2,PGA TOUR 2K23,PIPE by BMX Streets,PUBG: BATTLEGROUNDS,Palworld,"Papers, Please",Party Animals,Path of Exile,People Playground,Persona 3 Reload,Persona 4 Golden,Persona 5 Royal,Plague Inc: Evolved,Planet Coaster 2,Plants vs. Zombies GOTY Edition,Portal 2,PowerWash Simulator,Project Zomboid,RIDE 3,RIDE 4,RIDE 5,Red Dead Redemption 2,Resident Evil 4,Ride 2,Riders Republic,RimWorld,Risk of Rain 2,Rooftops & Alleys: The Parkour Game,Russian Fishing 4,Rust,STAR WARS Jedi: Survivor™,STAR WARS™ Battl

In [10]:

nombre_game = "69 Balls"
rank = 1
similar_ratings = matr_corre[nombre_game]*(rank-2.5)
recommendacion = similar_ratings.sort_values(ascending=False)

# printemoas la recomendacion
print(recommendacion[1:11].sort_values(ascending=False))

game_name
Automobilista 2                          0
Alien: Isolation                         0
Balatro                                  0
Armello                                  0
Apex Legends™                            0
Assetto Corsa Competizione               0
American Truck Simulator                 0
Age of Empires IV: Anniversary Edition   0
Ara: History Untold                      0
Age of Wonders 4                         0
Name: 69 Balls, dtype: float64
